In [3]:
import sys
sys.path.append("../..")

In [26]:
import os
import pickle

import pandas as pd
from pandas import DataFrame

from app.components.extract_data.extract_data import getDfPerson, getDfConsumo, getDictV9001toGroupPt, getDictV9001toGroupEn
from app.components.simple_types import Nutrition, State

In [11]:
datasetPath = os.getcwd() + "/../../datasets/"
datasetPicklePath = os.getcwd() + "/../../datasets/pickle/"

In [13]:
dfConsumo = getDfConsumo()
dfPerson = getDfPerson()

In [19]:
len(dfPerson["ESTRATO_POF"].unique())

575

In [37]:
class DietAgent:
    def __init__(self, person:str):
        self.initialDiet:State = State.getStateByPersonId(person)
        self.population:list[State] = [self.initialDiet]
        
        

In [ ]:
class HealthAgent:
    def __init__(self):
        self.personIds:list[str] = []
        self.solutions:list[State] = []
        self.dieatAgents:list[DietAgent] = []
        
    def add_person(self, person:str):
        personDiet = State.getStateByPersonId(person)        
        
        self.personIds.append(person)
        self.solutions.append(personDiet)
        self.dieatAgents.append( DietAgent(person) )
        

In [39]:
personID = "110000016#7#1#5"

In [42]:
healthAgent = HealthAgent()
dietAgent = DietAgent(personID)

In [43]:
healthAgent.add_person(personID)